<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install optuna

In [2]:
import optuna
import matplotlib.pyplot as plt
import numpy as np

import random

In [3]:
study = optuna.create_study(storage='sqlite:///model_selection.db', study_name='model_selection', direction='minimize', load_if_exists=True)

[I 2020-09-02 21:29:30,643] A new study created with name: model_selection


In [4]:
def f(x, y):
    return x**2 + y**2

In [5]:
class MyException(Exception):
    pass

def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    y = trial.suggest_uniform('y', -10, 10)
    if random.random() < 0.1:
        raise MyException()
    return f(x, y)

In [6]:
study.optimize(objective, n_trials=1000, catch=(MyException,))

[I 2020-09-02 21:29:49,472] Trial 0 finished with value: 11.107522601988208 and parameters: {'x': 3.3114475009802646, 'y': -0.37661392730456456}. Best is trial 0 with value: 11.107522601988208.
[I 2020-09-02 21:29:49,523] Trial 1 finished with value: 55.715658855759195 and parameters: {'x': -1.6646255461260857, 'y': 7.2763095485928595}. Best is trial 0 with value: 11.107522601988208.
[I 2020-09-02 21:29:49,570] Trial 2 finished with value: 120.99264333107769 and parameters: {'x': -5.072344835016896, 'y': -9.760325876002046}. Best is trial 0 with value: 11.107522601988208.
[W 2020-09-02 21:29:49,592] Trial 3 failed because of the following error: MyException()
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 709, in _run_trial
    result = func(trial)
  File "<ipython-input-5-4400ffc5829a>", line 8, in objective
    raise MyException()
MyException
[I 2020-09-02 21:29:49,666] Trial 4 finished with value: 34.12615153338112 and parame

KeyboardInterrupt: ignored

In [ ]:
study.best_params

In [ ]:
study.best_trial

In [ ]:
xs = [trial.params['x'] for trial in study.get_trials()]
ys = [trial.params['y'] for trial in study.get_trials()]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
ax.scatter(x=xs, y=ys, s=3, c=np.linspace(0, 1, 1_100))

# Visualizzazione

In [ ]:
study = optuna.load_study(study_name='model_selection', storage='sqlite:///model_selection.db' )

In [ ]:
optuna.visualization.plot_contour(study)